In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from copy import copy, deepcopy
from scipy import special
import pandas as pd
import matplotlib as mpl
from matplotlib.colors import colorConverter
import time
import pystencils
import sympy
from lbmpy.session import *
import math

In [ ]:
### CONSTANTS

# The value of the ambient lighting, i.e., when there's a shadow. Should be
#   0 <= ambient_strenght <= 1
AMBIENT_STRENGTH = 0.25
# The value of cells that have direct sunlight. Should be
#   0 <= sunlight_strength <= 1
SUNLIGHT_STRENGTH = 1.0

In [ ]:
### SUNLIGHT CONCENTRATIONS

def spawn_lighty_lighty(N):
    """
        Computes the initial sunlight matrix, i.e., sets everything to the
        sunlight value
        
        Parameters:
          - N: The width/height of the box we simulate
          
        Returns:
        An N x N matrix describing the sunlight concentrations for the first
        timestep
    """
    
    return np.array([[SUNLIGHT_STRENGTH for _ in range(N)] for _ in range(N)])

def growth_lighty_lighty(new_pixel, sunlight_mat, copy=True):
    """
        Given the location of the new pixel and the existing sunlight matrix,
        we update the sunlight matrices to include the new shadow
        
        Parameters:
          - new_pixels: tuple of (y, x) coordinates for the new pixel
          - sunlight_mat: An N x N matrix with the old sunlight concentrations
          - copy: If set to True, does not change the sunlight_mat directly but
            instead returns a new copy. (Default: True)

        Returns:
        An N x N matrix with new new sunlight concentrations yaaay
    """
    
    # Copy the sunlight matrix if needed
    if copy:
        sunlight_mat = sunlight_mat.copy()
    
    # Unpack the pixel coordinates
    y, x = new_pixel
    
    # Set the pixel to 0, to indicate that it's occupied with coral
    sunlight_mat[y, x] = 0
    
    # Loop down from there to 'cast' the shadow
    for iy in range(new_pixel[0] - 1, -1, -1):
        # If we happen to find a pixel/ambient strength, then we can stop early
        if sunlight_mat[iy, x] != SUNLIGHT_STRENGTH:
            break
        
        # Otherwise, cast the shadow
        sunlight_mat[iy, x] = AMBIENT_STRENGTH
    
    # We're done
    return sunlight_mat

def destroy_lighty_lighty(N, removed_coords, sunlight_mat, object_mat, copy = True):
    """
        Given lists of pixels that are removed, recomputes the shadow on their
        columns.
        
        Parameters:
          - N: The width/height of the box we simulate
          - removed_coords: List of (y, x) tuples describing each of the pixels
          - sunlight_mat: N x N matrix describing the sunlight concentrations
            at the previous step.
          - object_mat: N x N matrix of 1's and 0's describing where the coral
            lives. Assumes that the removed pixels have already been removed.
          - copy: If set to True, does not change the sunlight_mat directly but
            instead returns a new copy. (Default: True)
        
        Returns:
        An N x N matrix describing the sunlight at the next timestep
    """
    
    # Copy the sunlight matrix if needed
    if copy:
        sunlight_mat = sunlight_mat.copy()
        
    # Find the unique columns
    columns = set({ coord[1] for coord in removed_coords })
    
    # Loop thru the columns
    for x in columns:
        # Loop thru the column itself, spreading sunlight where we go
        for y in range(N - 1, -1, -1):
            # If it's a coral, then we can assume the rest is still shadow
            if object_mat[y, x] == 1:
                break
            
            # Otherwise, set the big sunlight
            sunlight_mat[y, x] = SUNLIGHT_STRENGTH
    
    # We're done already
    return sunlight_mat

In [ ]:
### FLOW CONCENTRATIONS

def velocity_info_callback(boundary_data, activate=True, **_):
    """
        Function to be able to activate and deactivate inflow speed
        
        Parameters:
          - boundary_data: the data for the lbmpy package that defines the
            boundary conditions
          - activate: whether or not the flow is activated
    """
    boundary_data['vel_1'] = 0
    if activate==True:
        u_max = 0.05
        boundary_data['vel_0'] = u_max 
    else:
        boundary_data['vel_0'] = 0

def figure_coral_with_flow(scenario, mode="vector"):
    """
        Function to plot the coral flow, either as vector field or as density
        field.
        
        Parameters:
          - scenario: LBMpy scenario describing the current flow
          - mode: Draw mode, either 'vector' for vectors or 'scalar' for
            density (default: vector).
        
        Returns:
        Nothing, but does show a pyplot.
    """

    plt.figure(dpi=200)
    if mode == "vector":
        plt.vector_field(scenario.velocity_slice());
    elif mode == "scalar":
        plt.scalar_field(scenario.velocity[:,:,0])
    plt.colorbar()
    plt.show()

# function to grow the object (by one) and to update the flow (lbm, ten steps)    
def grow_and_flow(object_mat, N, eta, nutrients_matrix, candidates, scenario, snapshots_vel, i, copy=True):
    """
        Function to grow / destroy the object and update the flow with ten
        steps.
        
        Parameters:
          - object_mat: N x N matrix describing where the coral lives.
          - N: The width/height of the box we simulate
          - eta: Some magical constant
          - nutrients_mat: Matrix describing the nutrient concentrations at the
            previous timestep
          - candidates: List of nodes we might grow to
          - scenario: LBMpy scenario describing the flow at the previous
            timestep
          - snaptshots_vel: List of snapshots of velocities, for the movies
          - i: Idk is this a loop??
          - copy: If set to True, copies the given object_mat and candidates
            before altering them. (Default: True)
        
        Returns:
        A tuple of:
          - The object matrix (copy), with a new growth / descruction
          - A new list of candidates (copy)
          - The given LBM scenario
          - The list of snapshots of the velocity, for el movie
    """
    
    # copy the object first if so desired
    if copy:
        object_mat = object_mat.copy()
        candidates = candidates.copy()
    
    # grow (and potentially destroy) new pixel(s)
    object_mat, candidates = choose_growth(N, eta, nutrients_matrix, candidates, object_mat)
    
    # set noslip boundary around object
    for j in range(N-1):
        for i in range(1,N-1):
            if object_mat[i][j] == 1:
                flag = scenario.boundary_handling.set_boundary(NoSlip(), make_slice[j,i])
    
    # run lbm 10 steps, saving at each step for movie
    for _ in range(10):
        scenario.run(1)
        snappy = deepcopy(scenario.velocity[:,:,0])
        snapshots_vel.append(snappy)
    return object_mat, candidates, scenario, snapshots_vel



def DLA_to_solution_nut(N, eta = 1, omega = 0.7, force=1e-4, duct=True, method='srt', relaxation_rate=1.9):
    """
        
        
        Parameters:
          - N: The width/height of the box we simulate
          - eta: Some math shit
          - omega: SOR parameter that determines how much to over/under relax
          - force: The power of the left inflow force
          - duct: Whether or not the simulation is a duct or not
          - method: Method of computation?
          - relaxation_rate: Basically the viscosity of the fluid.
          
        Returns:
        TBD
    """
    
    # initiate and advance a channel with lattice Boltzmann
    scenario = create_channel(
        domain_size = (N, N),
        force = force,
        duct = duct,
        method = method,
        relaxation_rate = relaxation_rate
    )

    # in- and outflow boundary
    stencil = get_stencil("D2Q9")
    outflow = ExtrapolationOutflow(stencil[4], scenario.method)
    scenario.boundary_handling.set_boundary(outflow, make_slice[:,N])
    inflow = UBB(velocity_info_callback, dim=scenario.method.dim)
    scenario.boundary_handling.set_boundary(inflow, make_slice[0, :])

    # keep track whether flow is on or of
    activ = True

    # list to save the snapshots
    snapshots_vel = []

    # 50 lbm steps, saving a snapshot of each flow update
    for _ in range(50):
        scenario.run(1)
        snappy = deepcopy(scenario.velocity[:,:,0])
        snapshots_vel.append(snappy)

    # initialisation of array with seed of object
    object_mat = np.zeros((N, N))
    object_mat[0][int(N / 2)] = 1
    candidates = set()
    candidates = get_candidates((0, int(N / 2)), object_mat, candidates)

    # initiation of the nutrient matrix
    nutrients_matrix = np.zeros((N, N))
    nutrients_matrix[:, 0:5] = 1

    # diffusion + convection for x timesteps
    snapshots_grad = []
    for iteration in range(2000):
        # TODO: ?
        # every 200 iterations, grow object, set noslip boundary and run LBM
        if iteration % 50 == 49:
            if activ == True:
                scenario.boundary_handling.trigger_reinitialization_of_boundary_data(activate=False)
                activ = False
            elif activ == False:
                scenario.boundary_handling.trigger_reinitialization_of_boundary_data(activate=True)
                activ = True
            
            # Update the nutrients 5 steps at a time
            for _ in range(5):
                object_mat, candidates, scenario, snapshots_vel = grow_and_flow(object_mat, N, eta, nutrients_matrix, candidates, scenario, snapshots_vel, iteration)   

            # Uncomment to plot the evolution occasionally
#             figure_coral_with_flow(scenario)
#             plot_object_gradient(nutrients_matrix, object_mat, 1)

        # every step, update the nutrient concentration matrix
        diffu_update = diffusion_update(nutrients_matrix, omega)
        conve_update = convection_update(nutrients_matrix, scenario)
        nutrients_matrix = nutrients_matrix + conve_update + diffu_update
        
        # check if nutrient concentration does not become negative
        for i in range(N):
            for j in range(N):
                if nutrients_matrix[i][j] < 0:
                     nutrients_matrix[i][j] = 0
    # TODO: check if this needs to stay
    #             if math.isnan(nutrients_matrix[i][j]):
    #                 print(nutrients_matrix)  # temporarily turned off for speed
                # concentration is 0 at object
                if object_mat[i][j] == 1:
                    nutrients_matrix[i][j] = 0
        
        # Bring new inflow of nutrients, and set the other boundries to be
        #   taking them away
        nutrients_matrix[:, 0:int(N / 10)] = 0.3
        nutrients_matrix[:,-1] = 0
        nutrients_matrix[0,:] = 0
        nutrients_matrix[-1,:] = 0
        snapshots_grad.append(nutrients_matrix)
    
    # When done, return the nutrients, 
    
    # When done, plot the final image
    plot_object_gradient(nutrients_matrix, object_mat, 1)

In [ ]:
### NUTRIENT CONCENTRATIONS

In [ ]:
### EROSION

def coral_breaky_breaky(N, seed_coord_x, threshold, object_mat, vector_field, sunlight_mat, copy=True):
    """
        Function that computes if the coral is gonna breaky breaky
        
        Parameters:
          - N: The width/height of the box we simulate in
          - seed_coord_x: the x-coordinate of the seed of the coral (the y is
            assumed to be 0)
          - threshold: The maximum force before a coral block erodes away
          - object_mat: a numpy array of 0's and 1's that determine where
            the coral is
          - vector_field: (I assume?) een numpy array of 2D vectors
          - sunlight_mat: N x N matrix describing the sunlights, which will be
            updated in case a pixels breaks.
          - copy: If True, does not modify the original object but instead
            returns a new one
          
        Returns:
          - The coral matrix with the relevant pixels removed
          - Broken boolean: True if part of coral got removed, else False
          
        O.O does it work?
    """
    
    # Copy the matrix if the user so desires
    if copy:
        object_mat = object_mat.copy()

    # Keep track whether part of coral has broken off
    broken = False

    # Loop through the coral matrix to find the corals
    for y in range(N):
        for x in range(N):
            if (x == seed_coord_x and y == 0) or object_mat[y, x] == 0: continue

            # Compute the pressure at this point (i.e., length of the vector)
            pressure = math.sqrt(vector_field[y, x][0] ** 2 + vector_field[y, x][1] ** 2)

            # If the pressure exceeds the threshold, remove the coral (:()
            if pressure > THRESHOLD:
                object_mat[y, x] = 0
                # Recompute the sunlight for this removed pixel
                destroy_lighty_lighty(N, [(y, x)], sunlight_mat, object_mat, False)
                broken = True

    # We're done! Return the results
    return object_mat, broken

def coral_painty_painty(seed_coord_x, object_mat, sunlight_mat, copy=True):
    """
        Function that checks which pixels are still connected to the source,
        and removes them. Also returns a new list of potential growth
        candidates.
        
        Note: We assume that a diagonal connection == no connection
        
        Parameters:
          - seed_coord_x: the x-coordinate of the seed of the coral (the y is
            assumed to be 0)
          - object_mat: a numpy array of 0's and 1's that determine where
            the coral is
          - sunlight_mat: N x N matrix describing the sunlights, which will be
            updated in case a pixels breaks.
          - copy: If True, does not modify the original object but instead returns a new one
        
        Returns:
        A tuple of:
          - The coral matrix, with all the unconnected pixels removed
          - A new list of growth candidates
    """
    
    # Copy the matrix if the user so desires
    if copy:
        object_mat = object_mat.copy()
        
    # Do a breadth-first search starting at the seed to see which pixels are connected to the seed
    object_mat[0, seed_coord_x] = 2
    to_do = [(0, seed_coord_x)]
    candidates = set()
    while len(to_do) > 0:
        # Fetch the pixel to check
        y, x = to_do[0]
        to_do = to_do[1:]
        
        # Get the area around the pixel
        for neighbour in [(-1, 0), (0, 1), (1, 0), (0, -1)]:
            ny = y + neighbour[0]
            nx = x + neighbour[1]

            # Skip if the pixel is out-of-bounds
            if nx < 0 or nx > object_mat.shape[0] - 1 or ny < 0 or ny > object_mat.shape[1] - 1:
                continue

            # If the pixel is not a pixel, then store it as possible growth candidate
            if object_mat[ny, nx] == 0:
                # Uncomment for the correct candidates order
#                 candidates.add((nx, ny))
                candidates.add((ny, nx))
            
            # If it is an (unvisited) pixel, then mark as visited/connected and add it to the todo list
            if object_mat[ny, nx] == 1:
                # Mark the pixel as connected
                object_mat[ny, nx] = 2
                
                # Add to the queue
                to_do.append((ny, nx))
    
    # Fetch the coordinates of the 1's
    removed_pixels = np.where(object_mat == 1)
    removed_coords = list(zip(*removed_pixels))
    
    # Go thru the matrix again and remove anything that's a 1
    object_mat[object_mat == 1] = 0
    # Convert the visited pixels back to 1's
    object_mat[object_mat == 2] = 1
    
    # Update the sunlight matrix to get rid of the ol' shadows
    destroy_lighty_lighty(N, removed_coords, sunlight_mat, object_mat, False)
    
    # Done!
    return object_mat, candidates

In [ ]:
### GAME LOOP

def get_candidates(object_loc, object_mat, candidates):
    """
        Function to find the neighbours of an object cell,
        if they qualify as growth candidates, add to set 
        (so all candidates are unique).
        
        Parameters:
          - object_loc: coordinates tuple of new object cell
          - object_mat: a numpy array of 0's and 1's that determine where
            the object is
          - candidates: a set of tuples containing the candidate coordinates
        
        Returns:
        The updated candidate set
    """

    # Get object coordinates
    y = object_loc[0]
    x = object_loc[1]

    # check if edirect neighbours are NOT part of object, and add them to candidates
    if x != N - 1 and object_mat[y][x + 1] == 0:
        candidates.add((y, x + 1))
        
    if x != 0 and object_mat[y][x - 1] == 0:
        candidates.add((y, x - 1))
        
    if y != N - 1 and object_mat[y + 1][x] == 0:
        candidates.add((y + 1, x))
        
    if y != 0 and object_mat[y - 1][x] == 0:
        candidates.add((y - 1, x))

    return candidates


def SOR_DLA_to_solution(N, eta, omega, iterations):
    """
        Function to calculate the SOR of a grid with object, 
        until convergence, with growing object
        
        Parameters:
          - N: desired grid size (NxN)
          - eta: weight of concentration gradient for growth
          - omega: SOR equation constant
          - iterations: how many times the object should grow
        
        Returns:
        - The concentration matrix
        - The object_mat matrix with fully grown object
        - The densitity list with coral density at each iteration
    """
    
    # Compute the source location (middle)
    seed_coord_x = int(N / 2)

    # Keep track of coral density at each iteration
    density_list = []
    
    # Initalisation of matrix with seed of object
    object_mat = np.zeros((N, N))
    object_mat[0, seed_coord_x] = 1
    candidates = set()
    candidates = get_candidates((0, seed_coord_x), object_mat, candidates)
    
    # First SOR to initialize starting concentrations
    sunlight_mat = spawn_lighty_lighty(N)
    # TODO: init nut matrix

    # loop until object is grown 'iterations' times, recomputing the sunlight with each growth
    for _ in range(iterations):
        # Grow the coral
        new_pixel, object_mat, candidates = choose_growth(N, eta, sunlight_mat, nut_mat, candidates, object_mat)
        
        # TODO: calc lattice boltzmann vector field
        sunlight_mat = growth_lighty_lighty(new_pixel, sunlight_mat, copy=False)
        # TODO: calc new nut_mat
        
        # TODO: (vector field)
        _, broken = coral_breaky_breaky(N, seed_coord_x, threshold, object_mat, vector_field, sunlight_mat, copy=False)
        
        if broken:
            _, candidates = coral_painty_painty(seed_coord_x, object_mat, sunlight_mat, copy=False)
            
        density_list.append(coral_density(object_array))
#     plt.imshow(conc_mat, origin='lower', extent=[0, 1, 0, 1], cmap='Spectral')
    return sunlight_mat, object_mat, density_list



def choose_growth(N, eta, sunlight_mat, nut_mat, candidates, object_mat):
    """
        Function to calculate growth probabilities of each candidate cell,
        choose one and grow it this timestep.
        
        Also breaks the coral due to errosion.

        Parameters:
          - N: grid size (NxN)
          - eta: weight of concentration gradient for growth
          - sunlight_mat: numpy array containing all sunlight concentrations
            per coordinate
          - candidates: set of tuple coordinates of all growable cells 
          - object_mat: N x N array of 1's and 0's describing where the color
            is

        Returns:
        - Location of the new pixel as (y, x) tuple
        - Updated object_mat matrix with newly grown object cell
        - Updated candidate set
    """
    
    # TODO: add necessary input constants as arguments (nutrient diff)
    
    probs = []  # store all candidate grow probabilities
    list_candidates = list(candidates)  # ensure same ordering 
    
    # Normalize the concentrations to turn them into probabilities
    for i in candidates:
        prob_sun = (sunlight_mat[i] ** eta) / np.sum([sunlight_mat[cand] ** eta for cand in candidates])
        # TODO: calculate nutrient probability and combine
        # prob_nut =
        # probs.append(alpha * prob_sun + (1 - alpha) * prob_nut)
        probs.append(prob_sun)

    # choose a candidate and grow
    chosen_growth = list_candidates[np.random.choice(len(candidates), p=probs)]
    object_mat[chosen_growth] = 1
    sunlight_mat[chosen_growth] = 0  # set conc to 0 where object has grown
    # update candidate set after growth
    candidates = get_candidates(chosen_growth, object_mat, candidates)
    # delete newly grown cell from growth candidates
    candidates.remove(chosen_growth)

    # Possibly break the coral
    
    return chosen_growth, object_mat, candidates

In [ ]:
### DENSITY / ANALYSIS
def coral_density(seed_coord_x, object_mat):
    """
        Function that computes the average distance per pixel to the source
        for the entire coral.
        
        Parameters:
          - seed_coord_x: the x-coordinate of the seed of the coral (the y is
            assumed to be 0)
          - object_mat: a numpy array of 0's and 1's that determine where
            the coral is
        
        Returns:
        The average distance of the coral. The lower, the denser.
    """
    
    # Search through the coral
    total_distance = 0
    n_pixels = 0
    for y in range(len(object_mat)):
        for x in range(len(object_mat[y])):
            # If not a coral or the source block, then skip
            if (x == seed_coord_x and y == 0) or object_mat[y][x] != 1: continue

            # If coral, then compute the distance to the source block
            total_distance += math.sqrt((x - seed_coord_x)**2 + y**2)
            n_pixels += 1

    # To return the average distance, we return total / count
    return total_distance / n_pixels

In [ ]:
### PLOTTING

# function to make a combined imshow plot, where the object is visible along with the gradient

## code (with small adjustments) based on answer at: https://stackoverflow.com/questions/10127284/overlay-imshow-plots-in-matplotlib
def plot_object_gradient(conc_mat, object_mat, eta):
    # generate the colors for your colormap
    color1 = colorConverter.to_rgba('white')
    color2 = colorConverter.to_rgba('black')

    # make the colormaps
    cmap2 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap2', [color1,color2], 256)
    cmap2._init() # create the _lut array, with rgba values

    # create your alpha array and fill the colormap with them.
    # here it is progressive, but you can create whathever you want
    alphas = np.linspace(0, 0.8, cmap2.N + 3)
    cmap2._lut[:, -1] = alphas

    img2 = plt.imshow(conc_mat, interpolation='nearest', cmap='Spectral', origin='lower', extent=[0, 1, 0, 1])
    plt.colorbar()
    img3 = plt.imshow(object_mat, interpolation='nearest', cmap=cmap2, origin='lower', extent=[0, 1, 0, 1])

    plt.title(f"Object with gradient, eta = {eta}")

    plt.show()
    
def amazing_graph(density_list):
    """
        Function that makes a graph of the density of a matrix, saves all intermidiate plots in folder named amazing_graph
        
        Parameters:
          - density_list 
        
        Returns:
        Fills fiel with graphs, make video with command :ffmpeg -i %04d.png -c:v libx264 -vf fps=2 -pix_fmt yuv420p test_out.mp4
        Make sure to be in path of amazing_graph
    """
    for i in range(len(density_list)):
        current = density_list[0:i]
#         plt.figure(dpi=300)
        plt.plot(current, color='green')
        plt.xlabel("Step number")
        plt.ylabel("Average distance to seed")
        plt.title("Density of Coral over step ")
    #     plt.show()

        plt.savefig(f'amazing_graph/{i:04d}.png')

In [ ]:
# Test run with just the sunlight computation

N = 100
eta = 1 
omega = 1.5
iterations = 100

conc_mat, object_mat, density_list = SOR_DLA_to_solution(N, eta, omega, iterations)
plot_object_gradient(conc_mat, object_mat, 1)
amazing_graph(density_list)

print(density_list)